In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
torch.cuda.device_count.cache_clear()

In [4]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/yelp/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [5]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/yelp/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
prompt_validation = []
recommend_validation = []
            
for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    for turn in row["turns"]:
        
        if "COMPUTER" in turn:
            computer = "Computer: " + turn["COMPUTER"] 
            recommend_validation.append(computer)
            prompt_validation.append(prompt)
            prompt += computer + "\n"
            
            if "HUMAN" in turn:
                prompt += "Human: " + turn["HUMAN"] + "\n"
            
tokenizer.truncation_side = 'left'  
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

100%|██████████| 2087/2087 [00:00<00:00, 18014.16it/s]


In [8]:
prompt_train = []
recommend_train = []

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    for turn in row["turns"]:
        
        if "COMPUTER" in turn:
            computer = "Computer: " + turn["COMPUTER"] 
            recommend_train.append(computer)
            prompt_train.append(prompt)
            prompt += computer + "\n"
            
            if "HUMAN" in turn:
                prompt += "Human: " + turn["HUMAN"] + "\n"
            

tokenizer.truncation_side = 'left'
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

100%|██████████| 9743/9743 [00:00<00:00, 19446.97it/s]


In [9]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [10]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/yelp/T5_responder",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=2, #gradient_accumulation_steps,
    per_device_train_batch_size=6, #train_batch_size,
    per_device_eval_batch_size=6, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss
13158,0.689500,0.640989
26316,0.630600,0.594914
39474,0.596000,0.575897


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [12]:
torch.cuda.empty_cache()

In [13]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/yelp/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [14]:
prompt_test = []
recommend_test = []
            
for _, row in tqdm(df_recommender_test.iterrows(), total=len(df_recommender_test)):
    prompt = ""
    for turn in row["turns"]:
        
        if "COMPUTER" in turn:
            computer = "Computer: " + turn["COMPUTER"] 
            recommend_test.append(computer)
            prompt_test.append(prompt)
            prompt += computer + "\n"
            
            if "HUMAN" in turn:
                prompt += "Human: " + turn["HUMAN"] + "\n"

100%|██████████| 2089/2089 [00:00<00:00, 19246.18it/s]


In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/yelp/T5_responder")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100
tokenizer.truncation_side = 'left'

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
bleu = evaluate.load("bleu")

In [17]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, bleu=bleu):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  predictions = []
  references = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=128,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, clean_up_tokenization_spaces=True).replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "") for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    predictions.extend(decoded_generations)
    references.extend([[gen] for gen in generation_batch])
    
    results = bleu.compute(predictions=predictions, references=references)

  return results

In [18]:
prompt_test

['',
 "Computer: Hi! How can I assist you today?\nHuman: I'm interested in a vegan place in Tampa.\n",
 "Computer: Hi! How can I assist you today?\nHuman: I'm interested in a vegan place in Tampa.\nComputer: Can you tell me which features you're looking for?\nHuman: I'm interested in locations that offer flavorful bowls.\n",
 "Computer: Hi! How can I assist you today?\nHuman: I'm interested in a vegan place in Tampa.\nComputer: Can you tell me which features you're looking for?\nHuman: I'm interested in locations that offer flavorful bowls.\nComputer: On a price scale from 1 to 4, with 1 being low and 4 being high, what is your price range?\nHuman: Price is not an issue.\n",
 "Computer: Hi! How can I assist you today?\nHuman: I'm interested in a vegan place in Tampa.\nComputer: Can you tell me which features you're looking for?\nHuman: I'm interested in locations that offer flavorful bowls.\nComputer: On a price scale from 1 to 4, with 1 being low and 4 being high, what is your price r

In [19]:
results = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, bleu=bleu)
print("results: ", results)

100%|██████████| 2698/2698 [2:03:24<00:00,  2.74s/it]

results:  {'bleu': 0.2277688594429611, 'precisions': [0.4185997620158852, 0.2578609842361885, 0.17544520848844145, 0.14211871212674307], 'brevity_penalty': 1.0, 'length_ratio': 1.2753358178942007, 'translation_length': 475662, 'reference_length': 372970}
